# Laboratorio: Recommendation con Surprise

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Recommendation

- I sistemi di _recommendation_ sono usati per dare agli utenti di un servizio dei suggerimenti mirati in base ai loro interessi
  - suggerire prodotti da acquistare su Amazon
  - suggerire film o serie da vedere su Netflix
  - suggerire canzoni da ascoltare su Spotify
  - ...
- I metodi di _collaborative filtering_ forniscono suggerimenti sulla base delle associazioni esistenti tra utenti e oggetti
  - ad es. nei metodi basati su similarità, per prevedere il voto che un utente $u$ darebbe ad un oggetto $i$ si fa la media (pesata)
    - dei voti dati ad $i$ da utenti che han dato voti simili ad $u$ ad altri oggetti (_user-based_)
    - dei voti dati da $u$ ad oggetti che hanno ricevuto voti simili ad $i$ da altri utenti (_item-based_)

## Surprise

- _Surprise_ è una libreria Python per la creazione e la validazione di modelli di recommendation
  - definisce strutture per rappresentare i dati su cui addestrare i modelli
  - permette di caricare dati da diverse fonti o di utilizzare dataset d'esempio
  - implementa diverse tecniche basate su similarità, scomposizione di matrici, ...
  - fornisce funzionalità per validare i modelli calcolando comuni metriche di accuratezza, ad es. RMSE

## Installazione di Surprise

- Seguire le istruzioni che seguono per installare Surprise nei PC di laboratorio
  - queste istruzioni sono valide per qualsiasi PC con Anaconda installato
- Aprire un prompt dei comandi o terminale
  - in Windows: tasto Windows -> `cmd` -> Invio
- Creare un ambiente Anaconda con Python e alcune librerie di supporto
  - `conda create -n surprise python=3.6 numpy pandas scipy joblib ipykernel`
- Attivare l'ambiente appena creato
  - Windows: `activate surprise`
  - Mac/Linux: `source activate surprise`
- Installare Surprise
  - `conda install -c conda-forge scikit-surprise`
- Aggiungere l'ambiente come kernel in Jupyter
  - `python -m ipykernel install --user --name surprise --display-name Surprise`

## Uso di Surprise

- Se questo file era già aperto in Jupyter, aggiornare la pagina
- Dal menù _Kernel_ selezionare _Change kernel_, quindi _Surprise_
- Eseguire il seguente import e verificare che non ci siano errori

In [1]:
import surprise

## Dataset

- Un `Dataset` consiste in un insieme di voti, ciascuno dato da un determinato utente ad un determinato oggetto
- Utenti e oggetti in un `Dataset` sono rappresentati con identificatori _raw_ arbitrari, spesso numeri o stringhe
- Un `Dataset` può essere ottenuto da un file CSV o da un DataFrame pandas (a sua volta ottenibile da diverse fonti)
- Surprise permette inoltre di caricare diversi dataset d'esempio di uso comune, scaricati _on demand_ dal Web
- Carichiamo ad esempio il dataset _MovieLens-100k_, con 100.000 voti dati da migliaia di utenti a migliaia di film

In [2]:
ml_data = surprise.Dataset.load_builtin("ml-100k")

## Trainset

- Un `Trainset` contiene le stesse informazioni di un `Dataset` in forma ottimizzata per l'utilizzo da parte degli algoritmi d'apprendimento
- $M$ utenti ed $N$ oggetti distinti sono identificati in un `Trainset` da identificatori interni (_inner_), ovvero numeri seriali da 0 a $M-1$ e da 0 a $N-1$
- Per creare un `Trainset` con tutti i voti contenuti in un `Dataset`, usare il metodo `build_full_trainset` di quest'ultimo

In [3]:
ml_train = ml_data.build_full_trainset()

## Estrarre Informazioni da un Trainset

- Da un `Trainset` abbiamo accesso rapido ad informazioni quali il numero di utenti distinti, oggetti distinti e voti complessivi

In [4]:
ml_train.n_users, ml_train.n_items, ml_train.n_ratings

(943, 1682, 100000)

- Possiamo consultare l'elenco dei voti noti di un qualsiasi utente od oggetto dai dizionari `ur` e `ir`
  - vanno usati gli ID seriali interni dal `Trainset`

In [5]:
# es.: 3 voti dati dall'utente 0
ml_train.ur[0][:3]
# ogni tupla: (ID oggetto, voto)

[(0, 3.0), (528, 4.0), (377, 4.0)]

- Possiamo vedere la media globale di tutti i voti dati

In [6]:
ml_train.global_mean

3.52986

- Possiamo usare i metodi `to_inner_uid` e `to_raw_uid` per convertire tra gli ID utenti originali del `Dataset` (_raw_) e gli ID utenti seriali del `Trainset` (_inner_)
- Possiamo fare lo stesso per gli ID oggetti con `to_inner_iid` e `to_raw_iid`

In [7]:
ml_train.to_raw_uid(0)

'196'

In [8]:
ml_train.to_inner_uid("196")

0

## User-based Collaborative Filtering

- Nella recommendation _user-based_, il voto $\hat{r}_{ui}$ previsto per un oggetto $i$ da parte di un utente $u$ è dato dalla media pesata dei voti dati ad $i$ dai $k$ utenti più simili ad $u$, rappresentati in un insieme $N_i^k(u)$
$$ \hat{r}_{ui} = \frac{\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}}{\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)} $$
- La similarità $\text{sim}(u,v)$ tra due utenti $u$ e $v$ è misurata dai voti dati ad oggetti recensiti da entrambi

## Addestramento di un Modello User-Based

- In modo simile a scikit-learn, creiamo dapprima un modello user-based "vuoto" impostando eventuali parametri
  - impostiamo ad esempio il numero massimo _k_ di utenti vicini da considerare

In [9]:
ubr = surprise.KNNBasic(k=150)

- Per addestrare il modello usiamo il metodo `fit` passando il `Trainset`
  - su IPython/Jupyter si può usare `%%time` per stampare il tempo necessario all'esecuzione

In [10]:
%%time
ubr.fit(ml_train)

Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 501 ms, sys: 13.1 ms, total: 514 ms
Wall time: 514 ms


## Utilizzare un Modello

- Una volta addestrato, il modello può prevedere il rating che un utente darebbe ad un oggetto dati i rispettivi ID "raw" (cioè quelli usati nel `Dataset` originale)
- Ad esempio, per conoscere il voto predetto per l'oggetto con ID "242" dall'utente con ID "196":

In [11]:
pred = ubr.predict("196", "242")
pred

Prediction(uid='196', iid='242', r_ui=None, est=3.949815800366104, details={'actual_k': 117, 'was_impossible': False})

- Otteniamo un oggetto `Prediction` i cui attributi riepilogano la richiesta (`uid` e `iid`) e forniscono i dati della predizione
- Il voto predetto è dato dall'attributo `est`, in questo caso circa 3,9 stelle
- I `details` indicano informazioni aggiuntive, in questo caso che il voto è stato predetto sulla base di quelli dati da 117 utenti simili al "196"
- Possiamo accedere a tutti i dati come attributi dell'oggetto, ad es.:

In [12]:
pred.est

3.949815800366104

## Validare un Modello

- Per valutare le prestazioni di un modello di recommendation e ricercare il migliore, come per i problemi di regressione, i dati a disposizione devono essere divisi in due set disgiunti
  - sul _training set_ viene addestrato il modello
  - sul _validation set_ le risposte del modello vengono confrontate con quelle reali
- Similmente a scikit-learn, Surprise offre un metodo `train_test_split` per dividere un `Dataset` in training e validation set
  - `test_size` e `random_state` funzionano come in scikit-learn

In [13]:
ml_train, ml_val = surprise.model_selection.train_test_split(ml_data, test_size=0.3, random_state=42)

- Il training set `ml_train` è un oggetto della classe `Trainset` vista sopra che stavolta contiene solo il 70\% dei 100.000 voti del dataset

In [14]:
ml_train.n_users, ml_train.n_items, ml_train.n_ratings

(943, 1638, 70000)

- Il validation set `ml_val` è costituito invece da un elenco di tuple `(id_utente, id_oggetto, voto)` non incluse nel training set

In [15]:
ml_val[:3]

[('657', '118', 1.0), ('846', '452', 3.0), ('620', '931', 3.0)]

- Le tuple contenute sono il restante 30\% del dataset

In [16]:
len(ml_val)

30000

## Ottenere le Predizioni per il Validation Set

- Una volta addestrato il modello sul training set...

In [17]:
%%time
ubr = surprise.KNNBasic(k=50)
ubr.fit(ml_train)

Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 265 ms, sys: 6.79 ms, total: 271 ms
Wall time: 273 ms


- ...usando il metodo `test`, possiamo passare l'elenco di tuple usato come validation set per ottenere un elenco di corrispondenti oggetti `Prediction`

In [18]:
%%time
ml_preds = ubr.test(ml_val)

CPU times: user 4.31 s, sys: 11 ms, total: 4.32 s
Wall time: 4.32 s


In [19]:
ml_preds[:3]

[Prediction(uid='657', iid='118', r_ui=1.0, est=3.2159872975510826, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='846', iid='452', r_ui=3.0, est=2.8217010882489393, details={'actual_k': 46, 'was_impossible': False}),
 Prediction(uid='620', iid='931', r_ui=3.0, est=2.2736204090846504, details={'actual_k': 42, 'was_impossible': False})]

- Ogni oggetto `Prediction` contiene sia il voto predetto (`est`) che quello reale dato dal validation set (`r_ui`)
- L'insieme di oggetti può così essere usato per valutare il recommender

## Valutazione dell'Accuratezza

- Lo scostamento globale tra voti reali e predetti del validation set può essere misurato in vari modi
- Quello più comune è il _Root Mean Squared Error_ (RMSE), in pratica la radice quadrata dell'errore quadratico medio già visto nella regressione
$$ \text{RMSE} = \sqrt{\frac{1}{|\hat{R}|} \sum_{\hat{r}_{ui} \in \hat{R}}(r_{ui} - \hat{r}_{ui})^2} $$
- Surprise implementa questa ed altre metriche nel modulo `accuracy`, alle quali va passato l'insieme di oggetti `Prediction` ottenuto da `test`

In [20]:
surprise.accuracy.rmse(ml_preds)

RMSE: 0.9917


0.9917209225966906

- Il risultato ottenuto può essere confrontato con modelli addestrati con diversi parametri o con altri algoritmi, per cercare quale funzioni meglio

## k-fold Cross-Validation

- Per fare in modo che il modello sia testato su tutti i dati, è comune usare la _k-fold cross-validation_
  - il dataset è diviso in k sottoinsiemi (_fold_) di pari dimensioni
  - ogni fold è usato come validation set su un modello addestrato sull'unione degli altri fold per ottenere una misura di accuratezza (es. RMSE)
  - l'accuratezza finale è la media delle misure sui singoli fold
- Per effettuare la divisione, Surprise offre una classe `KFold` che da un `Dataset` genera _k_ coppie di `Trainset` e validation set corrispondenti

- Ad esempio il seguente codice genera tre suddivisioni e per ciascuna addestra e valida un modello stampando il RMSE risultante

In [21]:
%%time
splitter = surprise.model_selection.KFold(n_splits=3, random_state=42)
for n, (train, val) in enumerate(splitter.split(ml_data)):
    print("FOLD {}".format(n+1))
    ubr = surprise.KNNBasic(k=50)
    ubr.fit(train)
    preds = ubr.test(val)
    surprise.accuracy.rmse(preds)

FOLD 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9874
FOLD 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9944
FOLD 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9940
CPU times: user 14.9 s, sys: 56.8 ms, total: 15 s
Wall time: 15 s


- Per eseguire rapidamente una k-fold cross validation, si può usare la funzione `cross_validate` specificando:
  - il modello (algoritmo e parametri) su cui eseguire la validazione
  - il `Dataset` da utilizzare
  - parametri opzionali quali numero di fold (`cv`, default 5), misure da calcolare (default RMSE e MAE), ...
- La funzione restituisce i risultati in un `dict`, che può essere incapsulato in un frame pandas per comodità

In [22]:
import pandas as pd
algo = surprise.KNNBasic(k=50)
cv_results = surprise.model_selection.cross_validate(algo, ml_data, cv=5)
cv_results = pd.DataFrame(cv_results)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


- Possiamo consultare i risultati dei singoli test dal frame, che includono anche i tempi impiegati per addestramento e valutazione...

In [23]:
cv_results

,test_rmse,test_mae,fit_time,test_time
0,0.984627,0.778647,0.414596,3.252371
1,0.984403,0.780505,0.315290,3.044279
2,0.976203,0.771734,0.319620,3.132861
3,0.981606,0.772728,0.313446,3.075123
4,0.981768,0.776023,0.313927,3.034144


- ...e usando le funzioni di pandas possiamo calcolare statistiche d'interesse, ad es. il RMSE medio

In [24]:
cv_results.test_rmse.mean()

0.9817214982408036

## Grid Search

- Come i modelli di regressione, anche quelli di recommendation hanno degli iperparametri da impostare che possono influenzarne l'accuratezza
  - ad esempio il numero k di vicini nella recommendation user-based
- Surprise offre una funzionalità di _grid search_ simile a quella di scikit-learn per testare diversi valori dei parametri
- Si definisce una "griglia" con i valori possibili dei parametri, ad esempio k

In [34]:
grid = {"k": [50, 100, 200]}

- Si crea quindi un modello `GridSearchCV` indicando la classe del modello da addestrare, la griglia dei parametri e il numero di fold di cross validation

In [34]:
gs = surprise.model_selection.GridSearchCV(surprise.KNNBasic, grid, cv=3)

- Si chiama quindi il metodo `fit` passando il dataset completo con i dati

In [35]:
gs.fit(ml_data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


- Il modello addestrato offre attributi simili alla grid search di scikit-learn
- Ad esempio `cv_results` fornisce risultati dettagliati sui parametri testati

In [33]:
pd.DataFrame(gs.cv_results)

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,...,split4_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k
0,0.980350,0.984918,0.982322,0.973918,0.984043,0.981110,0.003921,1,0.774746,0.779236,...,0.777062,0.775510,0.002954,1,0.309208,0.007968,3.196229,0.071187,{'k': 50},50
1,0.992046,0.996194,0.992895,0.985554,0.994411,0.992220,0.003620,2,0.785767,0.789788,...,0.786961,0.786002,0.002780,2,0.339286,0.057372,3.776299,0.090574,{'k': 100},100
2,1.002007,1.006249,1.002672,0.995517,1.004818,1.002253,0.003692,3,0.795665,0.800129,...,0.797001,0.796047,0.002928,3,0.301712,0.009867,4.033393,0.151193,{'k': 200},200


## Item-Based Collaborative Filtering

- La classe `KNNBasic` può essere usata anche per eseguire recommendation item-based, metodo duale allo user-based
- Il voto $\hat{r}_{ui}$ previsto per un oggetto $i$ da parte di un utente $u$ è dato dalla media pesata dei voti dati da $u$ ai $k$ oggetti più simili ad $i$, rappresentati in un insieme $N_u^k(i)$
$$ \hat{r}_{ui} = \frac{\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}{\sum\limits_{j \in N^k_u(j)} \text{sim}(i, j)} $$
- La similarità $\text{sim}(i,j)$ tra due oggetti $i$ e $j$ è misurata dai voti ricevuti da utenti che hanno recensito entrambi

- Per eseguire item-based recommendation al posto di user-based, cambiamo le opzioni per il calcolo della similarità in questo modo:

In [31]:
%%time
ibr = surprise.KNNBasic(k=50, sim_options={"user_based": False})
ibr.fit(ml_train)

Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 364 ms, sys: 36.4 ms, total: 400 ms
Wall time: 400 ms


In [32]:
%%time
surprise.accuracy.rmse(ibr.test(ml_val))

RMSE: 0.9838
CPU times: user 5.37 s, sys: 3.38 ms, total: 5.37 s
Wall time: 5.42 s


0.9837649345323103

## Collaborative Filtering con Fattorizzazione di Matrici

- I metodi basati su fattorizzazione o scomposizione di matrici funzionano rappresentando utenti ed oggetti come combinazione di fattori
  - i fattori sono ricavati statisticamente dai dati e corrispondono a grandi linee a categorie di oggetti (es. per i film: azione, commedia, ...)
  - ciascun oggetto è rappresentato da un vettore col peso di ciascun fattore su di esso (es. quanto un film è d'azione)
  - il vettore di ciascun utente indica l'affinità a ciascun fattore (es. quanto gli piacciono i film d'azione)
  - il voto stimato è quindi proporzionale alla similarità tra il vettore dell'utente e quello del prodotto
- Il modello di questo tipo più semplice è `SVD`
  - il parametro principale è il numero di fattori da individuare

- L'algoritmo ha una componente di casualità (la discesa gradiente stocastica), per ottenere risultati riproducibili è necessario impostare il seed tramite NumPy

In [33]:
import numpy as np
np.random.seed(42)

In [34]:
%%time
fbr = surprise.SVD(n_factors=100)
fbr.fit(ml_train)

CPU times: user 3.42 s, sys: 40 µs, total: 3.42 s
Wall time: 3.43 s


In [35]:
surprise.accuracy.rmse(fbr.test(ml_val))

RMSE: 0.9433


0.9432933101147625

## Recommendation Casuale

- Surprise offre anche un recommender che prevede voti casuali, utilizzabile come baseline nella valutazione degli altri metodi
  - i voti predetti hanno una distribuzione normale con media e varianza calcolate dal training set

In [36]:
%%time
rr = surprise.NormalPredictor()
rr.fit(ml_train)

CPU times: user 86.1 ms, sys: 0 ns, total: 86.1 ms
Wall time: 85.6 ms


In [40]:
%%time
surprise.accuracy.rmse(rr.test(ml_val))

RMSE: 1.5133
CPU times: user 290 ms, sys: 9.93 ms, total: 300 ms
Wall time: 298 ms


1.5132517686640503

- Estraendo i voti a caso ad ogni chiamata, la valutazione eseguita più volte da risultati diversi (se non si imposta un seed)
- Ovviamente questo metodo è molto veloce ma ha un RMSE molto più alto

## Recommendation su Dati Amazon

- Nelle prime esercitazioni avevamo visto un metodo di recommendation semplice applicato su dati di vendite estratti da Amazon
  - i dati riguardavano solamente i prodotti acquistati dai clienti, senza considerare i voti
- Riprendiamo ora gli stessi dati, ma includendo l'informazione sui voti dati dai clienti
- I dati sono divisi in due file CSV
  - uno contenente gli acquisti fino alla fine del 2000, da usare come training set
  - uno contenente gli acquisti dal 2001 in poi, da usare come validation set
- Scarichiamo i due file

In [42]:
import os.path
from urllib.request import urlretrieve
if not os.path.exists("amazon_train.csv"):
    urlretrieve("https://bit.ly/2LdmgTR", "amazon_train.csv")
if not os.path.exists("amazon_val.csv"):
    urlretrieve("https://bit.ly/2IPlyxO", "amazon_val.csv")

- Per prima cosa, creiamo un `Reader` impostando le caratteristiche dei file
  - scala dei voti da 1 a 5
  - virgola come separatore di campo

In [44]:
reader = surprise.Reader(rating_scale=(1, 5), sep=",")

- Leggiamo i due file in forma di `Dataset`

In [45]:
amazon_train_data = surprise.Dataset.load_from_file("amazon_train.csv", reader)
amazon_val_data = surprise.Dataset.load_from_file("amazon_val.csv", reader)

- Il primo va usato come training set, lo convertiamo quindi in oggetto `Trainset`

In [46]:
amazon_train = amazon_train_data.build_full_trainset()

- Nelle prime esercitazioni il file di training conteneva 9.683 acquisti di 3.384 prodotti distinti eseguiti da 178 prodotti distinti: verifichiamo che i numeri combacino

In [51]:
amazon_train.n_users, amazon_train.n_items, amazon_train.n_ratings

(178, 3384, 9683)

- Il secondo va invece convertito in una lista di tuple `(utente, oggetto, voto)` in questo modo:

In [47]:
amazon_val = amazon_val_data.build_full_trainset().build_testset()

- Il dataset contiene nomi di utenti e film, come possiamo vedere ad esempio dai primi 10 del validation set...

In [34]:
amazon_val[:10]

[('[1092996] Reviewer', '[4742] Nights of Cabiria [VHS]', 5.0),
 ('[1092996] Reviewer', '[50377] The Quiet Man [VHS]', 5.0),
 ('[1092996] Reviewer', '[51373] Home Alone 2 - Lost in New York [VHS]', 3.0),
 ('[1092996] Reviewer', '[96217] Dirty Harry [VHS]', 5.0),
 ('[1092996] Reviewer', '[57410] The Minus Man [VHS]', 3.0),
 ('[1092996] Reviewer', '[56285] Price Above Rubies [VHS]', 4.0),
 ('[1092996] Reviewer', '[5099] Brokedown Palace', 3.0),
 ('[1092996] Reviewer', '[42969] Stripes', 5.0),
 ('[1092996] Reviewer', '[96764] A Few Good Men [VHS]', 5.0),
 ('[1092996] Reviewer', '[96502] West Side Story [VHS]', 5.0)]

- Il numero di ulteriori voti contenuti in questo set è:

In [54]:
len(amazon_val)

5871

## Esercizi

Utilizzando `amazon_train` come training set e `amazon_val` come validation set

1. Calcolare il RMSE di un recommender casuale
2. Calcolare il RMSE di uno user-based recommender con k=50
3. Usando un recommender SVD, individuare per quale numero di fattori incluso in 2, 4, 6, ..., 20 si ottiene il miglior RMSE
  - usare un ciclo for o una list comprehension
  - fissare uno stesso seed prima dell'addestramento di ciascun modello
4. Usando SVD con gli stessi numeri possibili di fattori, eseguire una grid search sul dataset `amazon_train_data`
5. Usando il recommender al punto 2, stampare i nomi dei 10 film con voto stimato più alto da suggerire all'utente con ID 0 nel `Trainset`
  - per semplicità, non è necessario escludere film già valutati